In [49]:
#!pip install tensorflow==2.7.0
#!pip install tensorflow_addons

In [188]:
import tensorflow as tf

In [189]:
assert len(tf.config.list_physical_devices('GPU')) >= 1, 'Grouped Convs only work on TPU and GPU atm'

AssertionError: Grouped Convs only work on TPU and GPU atm

In [119]:
import tensorflow_addons as tfa
import tensorflow.keras.layers as layers

kernel_initial = tf.keras.initializers.TruncatedNormal(stddev=0.2)
bias_initial   = tf.keras.initializers.Constant(value=0)

def convnext(
    x,
    include_top            = True,
    weights                = 'imagenet',
    num_classes            = 1000,
    depths                 = [3, 3, 9, 3],
    dims                   = [96, 192, 384, 768],
    drop_path_rate         = 0.,
    classifier_activation  = 'softmax'
):
    
    assert len(depths) == len(dims)
    
    def forward_features(x):
        i = 0
        for depth, dim in zip(depths, dims):
            
            if i == 0:
                x = layers.Conv2D(
                    dim,
                    kernel_size = 4,
                    strides     = 4,
                    padding     = "valid",
                    name        = "downsample_layers.0.0_conv"
                )(x)
                x = layers.LayerNormalization(
                    epsilon  = 1e-6,
                    name     = "downsample_layers.0.0_norm"
                )(x)
            else:
                x = layers.LayerNormalization(
                    epsilon = 1e-6,
                    name    = "stages." + str(i) + "." + str(k) + ".downsample_norm"
                )(x)
                x = layers.Conv2D(
                    dim,
                    kernel_size  = 2,
                    strides      = 2,
                    padding            ='same',
                    kernel_initializer = kernel_initial,
                    bias_initializer   = bias_initial,
                    name = "stages." + str(i) + "." + str(k) + ".downsample_conv"
                )(x)
            
            
            for k in range(depth):
                x = add_convnext_block(
                    x,
                    dim,
                    drop_path_rate,
                    prefix = "stages." + str(i) + "." + str(k)
                )        
            i = i +1

        return x
    
    x = forward_features(x)
    
    if include_top:
        x = layers.GlobalAveragePooling2D(
            name='avg'
        )(x)
        x = layers.LayerNormalization(
            epsilon=1e-6,
            name ="norm"
        )(x)
        
        
        x = layers.Dense(
            num_classes,
            activation         = classifier_activation,
            kernel_initializer = kernel_initial,
            bias_initializer   = bias_initial,
            name               = "head"
        )(x)
    else:
        x = layers.GlobalAveragePooling2D(name='avg')(x)
        x = layers.LayerNormalization(epsilon=1e-6, name="norm")(x)
    
    return x

class gamma(tf.keras.layers.Layer):
    
        
    def __init__(self, const_val = 1e-6, dim = None, name=None, **kwargs):
        super(gamma, self).__init__(name=name)
        
        self.dim   = dim
        self.const = const_val * np.ones((self.dim))

    def call(self, inputs, **kwargs):
        return tf.multiply(inputs, self.const)
    
    def get_config(self):
        config = super(gamma, self).get_config()
        
        config.update({ "const": self.const, "dim": self.dim })
        
        return config


    

def add_convnext_block(y, dim, drop_prob=0, prefix=""):
    skip = y
        
    y = layers.DepthwiseConv2D(
        kernel_size=7,
        padding='same',
        name = f'{prefix}.dwconv'
    )(y)
    
    y = layers.LayerNormalization(
        epsilon=1e-6,
        name=f'{prefix}.norm'
    )(y)
    
    y = layers.Dense(
        4 * dim,
        name=f'{prefix}.pwconv1'
    )(y)
    
   
    y = layers.Activation(
        'gelu',
        name=f'{prefix}.act'
    )(y)
    
    y = layers.Dense(
        dim,
        name=f'{prefix}.pwconv2'
    )(y)
    
    y = gamma(
        const_val = 1e-6,
        dim       = dim,
        name      = f'{prefix}.gamma'
    )(y)
    
    y = tfa.layers.StochasticDepth(
        drop_prob,
        name = f'{prefix}.drop_path'
    )([skip, y])
    
    return y

In [204]:
depths_dims = dict(
    convnext_xtiny  = (dict(depths=[3, 3, 6, 3],    dims=[66, 132, 264, 528])),
    # archictectures from: https://github.com/facebookresearch/ConvNeXt
    # A ConvNet for the 2020s: https://arxiv.org/abs/2201.03545
    convnext_tiny   = (dict(depths=[3, 3, 9, 3],    dims=[96, 192, 384, 768])),
    convnext_small  = (dict(depths=[3, 3, 27, 3],   dims=[96, 192, 384, 768])),
    convnext_base   = (dict(depths=[3, 3, 27, 3],   dims=[128, 256, 512, 1024])),  
    convnext_large  = (dict(depths=[3, 3, 27, 3],   dims=[192, 384, 768, 1536])),
    convnext_xlarge = (dict(depths=[3, 3, 27, 3],   dims=[256, 512, 1024, 2048])),
)

def build_model_functional(name, shape=(32, 32, 3), weights=None, **kwargs):
    image_tensor   = layers.Input(shape=shape, name="input")
    network_output = convnext(image_tensor, **depths_dims[name], **kwargs)
    
    model = tf.keras.Model(
        inputs  = [image_tensor],
        outputs = [network_output],
        name    = name
    )
    
    if weights:
        if type(weights) == str and os.path.exists(weights):
            model.load_weights(weights)
            
    return model

In [205]:
model = build_model_functional(name="convnext_tiny", shape=(32, 32, 3), num_classes = 100)
print(model.summary())

Model: "convnext_tiny"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
downsample_layers.0.0_conv (Con (None, 8, 8, 96)     4704        input[0][0]                      
__________________________________________________________________________________________________
downsample_layers.0.0_norm (Lay (None, 8, 8, 96)     192         downsample_layers.0.0_conv[0][0] 
__________________________________________________________________________________________________
stages.0.0.dwconv (DepthwiseCon (None, 8, 8, 96)     4800        downsample_layers.0.0_norm[0][0] 
______________________________________________________________________________________

In [206]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
    loss      = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics   = ['accuracy']
)


### Test

In [150]:
import cv2
def load_test_image(image_size=(224, 224)):
    from skimage.data import astronaut
    img = astronaut() 
    img = cv2.resize(img, image_size)
    img = img / 255
    img = (img - np.array([0.485, 0.456, 0.406])) / \
        np.array([0.229, 0.224, 0.225])
    return img

In [151]:
randx = tf.random.uniform((10, 32, 32, 3))

try:
    model.predict(randx)
except tf.errors.UnimplementedError as e:
    print("Exception: {} \n".format(e) )
    print("Direct call works fine:")
    print(model(randx))
    

In [152]:
model(np.expand_dims(load_test_image((32, 32)), axis = 0))

<tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[1.67772896e-09, 3.78736729e-07, 1.03951114e-08, 3.56828309e-07,
        7.41135376e-03, 1.31371814e-06, 1.69506457e-05, 9.77349259e-07,
        4.72753158e-07, 1.70275666e-06, 3.81116138e-07, 1.47781111e-06,
        6.12693100e-07, 8.82412314e-06, 1.15726076e-01, 1.61077185e-09,
        5.75246428e-09, 7.84593925e-04, 2.55230025e-05, 8.81508466e-09,
        6.28227252e-04, 1.80769473e-06, 9.75112016e-07, 2.56940187e-03,
        5.46242634e-04, 9.87005933e-06, 1.52337903e-04, 4.34378344e-05,
        7.60342633e-09, 6.83674102e-07, 9.00742307e-05, 6.00366584e-05,
        3.95735498e-08, 1.32590894e-05, 2.44327210e-04, 8.65205875e-05,
        2.88297451e-06, 1.53806599e-04, 2.69001885e-05, 1.46093387e-10,
        2.51428588e-07, 2.41148973e-05, 6.84985006e-03, 1.81391933e-05,
        1.13398277e-10, 4.96826018e-04, 2.53328949e-06, 2.60750930e-05,
        2.91160650e-05, 3.35537351e-07, 3.98769726e-06, 3.03069129e-07,
        2.6470

### Serialize Model

In [165]:
model.save("x_tiny_convnext", save_format = "tf")

INFO:tensorflow:Assets written to: x_tiny_convnext/assets


In [166]:
# Recreate the exact same model purely from the file
recreated = tf.keras.models.load_model('x_tiny_convnext')
recreated.summary()

Model: "convnext_tiny"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
downsample_layers.0.0_conv (Con (None, 8, 8, 96)     4704        input[0][0]                      
__________________________________________________________________________________________________
downsample_layers.0.0_norm (Lay (None, 8, 8, 96)     192         downsample_layers.0.0_conv[0][0] 
__________________________________________________________________________________________________
stages.0.0.dwconv (DepthwiseCon (None, 8, 8, 96)     4800        downsample_layers.0.0_norm[0][0] 
______________________________________________________________________________________

In [170]:
import tempfile

MODEL_DIR   = tempfile.gettempdir()
version     = 1
export_path = os.path.join(MODEL_DIR, str(version))

print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite         = True,
    include_optimizer = True,
    save_format       = None,
    signatures        = None,
    options           = None,
    save_traces       = True
)

print('\nSaved model:')
!ls -l {export_path}

export_path = /var/folders/mb/b12gjxj101g07wyss7rz69dh0000gn/T/1

INFO:tensorflow:Assets written to: /var/folders/mb/b12gjxj101g07wyss7rz69dh0000gn/T/1/assets

Saved model:
total 7392
drwxr-xr-x  2 leandermelms  staff       64 Jan 16 22:44 assets
-rw-r--r--  1 leandermelms  staff   384582 Jan 16 22:47 keras_metadata.pb
-rw-r--r--  1 leandermelms  staff  3389036 Jan 16 22:47 saved_model.pb
drwxr-xr-x  4 leandermelms  staff      128 Jan 16 22:47 variables


In [171]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 32, 32, 3)
        name: serving_default_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['head'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 100)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          inputs: TensorSpec(shape

### Start TF Serving
```bash
docker run --rm -d \
-p 8501:8501 \
-p 8500:8500 \
--mount type=bind,source={export_path},target=/models/T \
-e MODEL_NAME=T \
-t tensorflow/serving
```

In [201]:
import json

test_image = np.expand_dims(load_test_image((32, 32)), axis = 0)
data       = json.dumps({"signature_name": "serving_default", "instances": test_image.tolist()})

print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))


Data: {"signature_name": "serving_default", "instances": ... 44035, -1.7205882352941175, -1.5081481481481482]]]]}


In [202]:
# !pip3.8 install -q requests

In [203]:
import requests

headers       = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/T:predict', data=data, headers=headers)

if not json_response.status_code == 404:
    predictions = json.loads(json_response.text)['predictions']
    print(predictions[0], np.argmax(predictions[0]))


[6.06438903e-07, 0.000356569275, 2.23858732e-09, 1.80161658e-06, 6.90477066e-07, 0.0382772759, 2.31505965e-05, 3.41183693e-07, 1.10117753e-05, 2.91179674e-07, 7.38370145e-05, 3.88432625e-07, 2.00873137e-06, 3.57008212e-05, 1.5023619e-07, 4.10727952e-10, 5.23551122e-08, 9.21707942e-06, 6.52472465e-07, 1.66186601e-05, 0.00124993257, 2.28760921e-08, 0.00503779296, 1.46546961e-08, 8.42234e-09, 1.65633537e-06, 1.52961693e-05, 1.45454869e-05, 0.0127485096, 0.000143969577, 4.3050577e-06, 0.000124381579, 9.78936896e-06, 0.0551269, 4.29131342e-06, 7.36179162e-09, 0.000651590759, 2.57197446e-08, 2.08448491e-05, 1.56616097e-05, 0.410003483, 0.000102210128, 5.07701407e-05, 2.59493174e-08, 2.79966206e-09, 3.18298385e-06, 7.66703749e-08, 1.13806074e-07, 0.000471267937, 3.61099737e-05, 0.000835027371, 0.000842510141, 5.00963142e-05, 0.00426228, 0.018978294, 7.81761808e-11, 0.00129558041, 9.31576722e-07, 6.45944383e-05, 8.48438688e-08, 1.11496784e-05, 0.00396742579, 2.41857e-09, 6.31034652e-07, 0.0002